# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

# Question 1 - What test is appropriate for this problem? Does CLT apply?

##### A hypothesis z-test is appropriate in this instance as we are comparing the observed difference between two proportions (a point estimate) to the null (no difference) and because n > 30. Categorical data differs from numerical data in that we use the sum of 'successes' divided by the 'n' of the sample instead of calculating a mean. In this way we can determine if the proportion of sucesses/failures for each group in the explanatory variable. We can then compute the z-test.
##### The Central Limit theorum does apply in categorical data for the sampling distribution of the sample proportion. Distribution is nearly normal.

# Question 2 - What are the null and alternate hypotheses?

##### Null Hypothesis :  :
##### Null Hypothesis: That there is no significant differenece between the proportion of calls recieved from CV's with white-sounding names and those with black-sounding names
##### Alternative Hypothesis :  :
##### Alternative Hypothesis: That there is a significant difference between the proportion of calls recieved from CV's with white-sounding names and the proportion of those with black-sounding names.

In [5]:
w = data[data.race=='w']
b = data[data.race=='b']

# QUESTION 3: Compute margin of error, confidence interval, and p-value.

# First, take a look at the structure of the data

In [6]:
# The variables that are relevant to the above questions are race & call
# Creating a new variable to isolate call and race
df = pd.DataFrame([data.call,data.race])
df = df.transpose() # Making date are race the columns not index
df.head(10)

,call,race
0,0,w
1,0,w
2,0,b
3,0,b
4,0,w
5,0,w
6,0,w
7,0,b
8,0,b
9,0,b


In [7]:
# Separating the dataframe into black and white
df_b = df[df.race == 'b']
df_w = df[df.race == 'w']
# Checking 'n' of each sample
len(df_b), len(df_w)

(2435, 2435)

In [8]:
# Calculating the proportions per racial group 
prop_b = ((sum(df_b.call))/(len(df_b.call)))
prop_w = ((sum(df_w.call))/(len(df_w.call)))
prop_b, prop_w # p-hats for both black and white

(0.06447638603696099, 0.09650924024640657)

# Computing the Margin of Error


In [9]:
# Finding standard error for proportions (different for hypothesis test)
se_CI = np.sqrt((prop_b*(1 - prop_b)/(len(df_b))) + (prop_w*(1 - prop_w) /(len(df_w))))
se_CI

0.0077833705866767544

In [10]:
# We are calculating at the 5% significance level so our critical value is 1.96
crit = 1.96
margin = abs(crit*se_CI)
print("The true population proportion lies +/- %0.6F around the point estimate" % margin)

The true population proportion lies +/- 0.015255 around the point estimate


# Computing the Confidence Interval

In [11]:
# Race is our explanatory varible and Call is our response variable
# Observed Difference
point_est = prop_w - prop_b
point_est

0.032032854209445585

In [12]:
CI = [ point_est + margin, point_est - margin]
print("The proportion is %0.6F +/- %0.5F " % (point_est,margin))
print("The proportion of CVs with white-sounding names that recieve a call is between %0.6F and %0.6F higher than the proportion of CVS with black-sounding names" % (CI[1],CI[0]))

The proportion is 0.032033 +/- 0.01526 
The proportion of CVs with white-sounding names that recieve a call is between 0.016777 and 0.047288 higher than the proportion of CVS with black-sounding names


The confidence interval above tells us that there is are higher number of calls received by CVs with white-sounding names than are received by CVs with black-sounding names.

# Calculating the p-value

##### Using the test statistic

In [13]:
# Setting the parameters assuming Ho is true
# Expected Difference of prop_w - prop_b
null = 0

#Create a pooled proportion as expected value of calls across black & white 
p_pool = (sum(data.call)/(len(data.call)))
p_pool

0.08049281314168377

In [14]:
# Calculating Standard Error using the pooled proportion (again assuming that H0 is true)
se_ht = np.sqrt((p_pool*(1 - p_pool)/(len(df_b))) + (p_pool*(1 - p_pool) /(len(df_w))))
se_ht

0.007796894036170457

In [15]:
# z-score is the observed (prop_w - prop_b) - expected (null) divided by standard error (pooled, computed for the CI above)
# Expected Difference of prop_w - prop_b

z = (point_est - null)/se_ht #standard error calculated in CI above
p_values = stats.norm.sf(abs(z))*2 #twoside
print("Z-score is equal to : %6.3F  p-value equal to: %6.7F" % (z,p_values))

Z-score is equal to :  4.108  p-value equal to: 0.0000398


As you can see above, we have a p-value of 0.0000398 < 0.025, and as such we can reject the null hypothesis and conclude that there is a significant difference between black-sounding and white-sounding names in the number of calls received.
Just for the sake of diligence, let's double check this answer using the z-test function from Statsmodels

In [16]:
from statsmodels.stats.weightstats import ztest
ztest = ztest(data.call[data.race == 'w'],data.call[data.race == 'b'], alternative = 'two-sided')
print("Z-score is equal to : %6.3F  p-value equal to: %6.7F" % (abs(ztest[0]),ztest[1]))

Z-score is equal to :  4.115  p-value equal to: 0.0000388


As shown above, we get an almost identical result. There is a p-value of 0.0000388 < 0.025, and as such we can reject the null hypothesis and conclude that there is a significant difference between black-sounding and white-sounding names in the number of calls received.

# QUESTION 4: Write a story describing the statistical significance in the context of the original problem.

# ANSWER 4:

### With a p-value of less than 0.05 and confidence interval that does not contain 0, we can conclude that there is a statistically significant difference between callback rates for black and white-sounding names.


# QUESTION 5: Does the analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

# ANSWER 5:

### No, the conclusion above does not imply that race is the most important factor in callback success, because it solely examines race. All other relevant factors like education and years of experience would need to be examined and compared in order to identify the *most important* one.